In [ ]:
pip install tokenizers datasets transformers

In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import login

# Log in to Hugging Face
login(token="")

In [17]:
# Hugging Face 'The Stack - V2' which contains 67.5 TB of source code files. Python is among the major languages represented containing 80.6 M rows.
# Efficiently loading only the 'Python' subset and 'streaming=True' to process data batch by batch.
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("bigcode/the-stack-v2", "Python", split="train", streaming=True)

# Inspect the dataset
print(dataset)

Resolving data files:   0%|          | 0/917 [00:00<?, ?it/s]

IterableDataset({
    features: ['blob_id', 'directory_id', 'path', 'content_id', 'detected_licenses', 'license_type', 'repo_name', 'snapshot_id', 'revision_id', 'branch_name', 'visit_date', 'revision_date', 'committer_date', 'github_id', 'star_events_count', 'fork_events_count', 'gha_license_id', 'gha_event_created_at', 'gha_created_at', 'gha_language', 'src_encoding', 'language', 'is_vendor', 'is_generated', 'length_bytes', 'extension'],
    num_shards: 9
})


In [18]:
for sample in dataset.take(5):
    print(sample)

{'blob_id': 'd44bbb217114c0831167824d694d57c29ab86665', 'directory_id': 'e3f3f911019ac126d01c056eafc7c3183107a5af', 'path': '/Traffic Sign Detection/all_signs_combined/src/predict.py', 'content_id': '19ed9a428015b625610be9930dfee35938fb451b', 'detected_licenses': [], 'license_type': 'no_license', 'repo_name': 'uncctrafficsigndetection/Traffic-Sign-Detection', 'snapshot_id': '595258766f865c4b3c628b002d7b93a774168a9b', 'revision_id': '3ff4be52357f4b6340fef94124f8c835ab66fd8a', 'branch_name': 'refs/heads/master', 'visit_date': datetime.datetime(2020, 4, 9, 20, 28, 33, 910961), 'revision_date': datetime.datetime(2018, 12, 5, 21, 29, 50), 'committer_date': datetime.datetime(2018, 12, 5, 21, 29, 50), 'github_id': 160574509, 'star_events_count': 0, 'fork_events_count': 0, 'gha_license_id': None, 'gha_event_created_at': None, 'gha_created_at': None, 'gha_language': None, 'src_encoding': 'UTF-8', 'language': 'Python', 'is_vendor': False, 'is_generated': False, 'length_bytes': 959, 'extension': 

In [ ]:
import os

# Set AWS credentials (Enter your own keys)
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

In [ ]:
pip install smart_open[s3] boto3 datasets

In [ ]:
import os
import boto3
from smart_open import open
from datasets import load_dataset

# Set up AWS session
session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"]
)
s3 = session.client("s3")

# Function to download file contents
def download_contents(row):
    blob_id = row["blob_id"]
    src_encoding = row["src_encoding"]
    s3_url = f"s3://softwareheritage/content/{blob_id}"

    with open(s3_url, "rb", compression=".gz", transport_params={"client": s3}) as fin:
        content = fin.read().decode(src_encoding)

    return {"content": content}

# Load the dataset
ds = load_dataset("bigcode/the-stack-v2","Python", split="train", streaming=True)

# Map the download function to the dataset
ds = ds.map(download_contents)

# Inspect the first sample
for row in ds:
    print(row["content"])
    break

In [22]:
# Save the first 10000 samples to a file
with open("python_code.txt", "w") as f:
    for i, row in enumerate(ds):
        if i >= 10000:  # Adjust the number of samples as needed
            break
        f.write(row["content"] + "\n")

In [46]:
# Load the original dataset
with open("python_code.txt", "r") as f:
    code_samples = f.readlines()

# Split the dataset into training and held-out sets
train_size = int(0.9 * len(code_samples))  # 90% for training
train_samples = code_samples[:train_size]
held_out_samples = code_samples[train_size:]

# Save the training set
with open("train_python_code.txt", "w") as f:
    f.writelines(train_samples)

# Save the held-out set
with open("held_out_python_code.txt", "w") as f:
    f.writelines(held_out_samples)

In [24]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Initialize BPE tokenizer
bpe_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
bpe_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Train the tokenizer
trainer = trainers.BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=10000)
bpe_tokenizer.train(files=["python_code.txt"], trainer=trainer)

# Save the tokenizer
bpe_tokenizer.save("bpe_tokenizer.json")

In [25]:
from tokenizers import Tokenizer

# Load the trained BPE tokenizer
bpe_tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

# Sample Python code
sample_code = """
def add(a, b):
    return a + b
"""

# Tokenize the sample code
output = bpe_tokenizer.encode(sample_code)
print("Tokens:", output.tokens)
print("IDs:", output.ids)

Tokens: ['def', 'add', '(', 'a', ',', 'b', '):', 'return', 'a', '+', 'b']
IDs: [4557, 4834, 16, 73, 20, 74, 4538, 4594, 73, 19, 74]


In [26]:
import time

# Load a large dataset
with open("python_code.txt", "r") as f:
    large_code = f.read()

# Measure tokenization time
start_time = time.time()
output = bpe_tokenizer.encode(large_code)
end_time = time.time()

print("Tokenization Time:", end_time - start_time, "seconds")

Tokenization Time: 42.46228647232056 seconds


In [27]:
from tokenizers import Tokenizer

# Load the trained BPE tokenizer
bpe_tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

In [28]:
vocab_size = bpe_tokenizer.get_vocab_size()
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 10000


In [29]:
# Load a dataset of Python code
with open("python_code.txt", "r") as f:
    code_samples = f.readlines()

# Tokenize each sample and count the number of tokens
total_tokens = 0
total_samples = len(code_samples)

for code in code_samples:
    output = bpe_tokenizer.encode(code)
    total_tokens += len(output.tokens)

# Calculate the average number of tokens per sentence
avg_tokens_per_sentence = total_tokens / total_samples
print("Average Tokens per Sentence:", avg_tokens_per_sentence)

Average Tokens per Sentence: 10.974699436946016


In [30]:
# Load a held-out dataset
with open("held_out_python_code.txt", "r") as f:
    held_out_code = f.read()

# Tokenize the held-out code
output = bpe_tokenizer.encode(held_out_code)

# Count the number of OOV tokens
oov_tokens = [token for token in output.tokens if token == "[UNK]"]
num_oov_tokens = len(oov_tokens)
total_tokens = len(output.tokens)

# Calculate the OOV rate
oov_rate = num_oov_tokens / total_tokens
print("OOV Rate:", oov_rate)

OOV Rate: 0.0


In [31]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Initialize WordPiece tokenizer
wordpiece_tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

# Use a pre-tokenizer to split input into words
wordpiece_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Define a trainer
wordpiece_trainer = trainers.WordPieceTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
    vocab_size=10000  # Adjust vocabulary size as needed
)

# Train the tokenizer
wordpiece_tokenizer.train(files=["train_python_code.txt"], trainer=wordpiece_trainer)

# Save the tokenizer
wordpiece_tokenizer.save("wordpiece_tokenizer.json")

In [32]:
from tokenizers import Tokenizer

# Load the trained tokenizers
bpe_tokenizer = Tokenizer.from_file("bpe_tokenizer.json")
wordpiece_tokenizer = Tokenizer.from_file("wordpiece_tokenizer.json")

In [33]:
print("BPE Vocabulary Size:", bpe_tokenizer.get_vocab_size())
print("WordPiece Vocabulary Size:", wordpiece_tokenizer.get_vocab_size())

BPE Vocabulary Size: 10000
WordPiece Vocabulary Size: 10000


In [34]:
def calculate_avg_tokens_per_sentence(tokenizer, file_path):
    with open(file_path, "r") as f:
        code_samples = f.readlines()

    total_tokens = 0
    total_samples = len(code_samples)

    for code in code_samples:
        output = tokenizer.encode(code)
        total_tokens += len(output.tokens)

    return total_tokens / total_samples

# Calculate for each tokenizer
bpe_avg_tokens = calculate_avg_tokens_per_sentence(bpe_tokenizer, "held_out_python_code.txt")
wordpiece_avg_tokens = calculate_avg_tokens_per_sentence(wordpiece_tokenizer, "held_out_python_code.txt")

print("BPE Average Tokens per Sentence:", bpe_avg_tokens)
print("WordPiece Average Tokens per Sentence:", wordpiece_avg_tokens)

BPE Average Tokens per Sentence: 10.09957276736494
WordPiece Average Tokens per Sentence: 12.96194184123484


In [35]:
def calculate_oov_rate(tokenizer, file_path):
    with open(file_path, "r") as f:
        held_out_code = f.read()

    output = tokenizer.encode(held_out_code)
    oov_tokens = [token for token in output.tokens if token == "[UNK]"]
    num_oov_tokens = len(oov_tokens)
    total_tokens = len(output.tokens)

    return num_oov_tokens / total_tokens

# Calculate for each tokenizer
bpe_oov_rate = calculate_oov_rate(bpe_tokenizer, "held_out_python_code.txt")
wordpiece_oov_rate = calculate_oov_rate(wordpiece_tokenizer, "held_out_python_code.txt")

print("BPE OOV Rate:", bpe_oov_rate)
print("WordPiece OOV Rate:", wordpiece_oov_rate)

BPE OOV Rate: 0.0
WordPiece OOV Rate: 0.00019510514678871283


In [36]:
from tokenizers import Tokenizer

# Load the trained tokenizer
bpe_tokenizer = Tokenizer.from_file("bpe_tokenizer.json")  # or "wordpiece_tokenizer.json"

In [37]:
# Sample Python code snippets
code_samples = [
    "def add(a, b):",
    "    return a + b",
    "def factorial(n):",
    "    if n == 0:",
    "        return 1",
    "    else:",
    "        return n * factorial(n - 1)",
    "# This is a comment",
    "print('Hello, World!')"
]

# Tokenize each sample and print the results
for code in code_samples:
    output = bpe_tokenizer.encode(code)
    print(f"Input: {code}")
    print(f"Tokens: {output.tokens}")
    print(f"Token IDs: {output.ids}")
    print("-" * 40)

Input: def add(a, b):
Tokens: ['def', 'add', '(', 'a', ',', 'b', '):']
Token IDs: [4557, 4834, 16, 73, 20, 74, 4538]
----------------------------------------
Input:     return a + b
Tokens: ['return', 'a', '+', 'b']
Token IDs: [4594, 73, 19, 74]
----------------------------------------
Input: def factorial(n):
Tokens: ['def', 'factor', 'ial', '(', 'n', '):']
Token IDs: [4557, 7227, 5589, 16, 86, 4538]
----------------------------------------
Input:     if n == 0:
Tokens: ['if', 'n', '==', '0', ':']
Token IDs: [4536, 86, 4562, 24, 34]
----------------------------------------
Input:         return 1
Tokens: ['return', '1']
Token IDs: [4594, 25]
----------------------------------------
Input:     else:
Tokens: ['else', ':']
Token IDs: [4742, 34]
----------------------------------------
Input:         return n * factorial(n - 1)
Tokens: ['return', 'n', '*', 'factor', 'ial', '(', 'n', '-', '1', ')']
Token IDs: [4594, 86, 18, 7227, 5589, 16, 86, 21, 25, 17]
----------------------------------

In [38]:
def visualize_tokenization(tokenizer, text):
    output = tokenizer.encode(text)
    tokens = output.tokens
    highlighted_text = text
    for token in tokens:
        highlighted_text = highlighted_text.replace(token, f"[{token}]")
    print(f"Input: {text}")
    print(f"Highlighted Tokens: {highlighted_text}")
    print(f"Tokens: {tokens}")
    print("-" * 40)

# Visualize tokenization for each sample
for code in code_samples:
    visualize_tokenization(bpe_tokenizer, code)

Input: def add(a, b):
Highlighted Tokens: [def] [[a]dd][(][a][,] [b][):]
Tokens: ['def', 'add', '(', 'a', ',', 'b', '):']
----------------------------------------
Input:     return a + b
Highlighted Tokens:     [return] [a] [+] [b]
Tokens: ['return', 'a', '+', 'b']
----------------------------------------
Input: def factorial(n):
Highlighted Tokens: [def] [factor][ial][(][n][):]
Tokens: ['def', 'factor', 'ial', '(', 'n', '):']
----------------------------------------
Input:     if n == 0:
Highlighted Tokens:     [if] [n] [==] [0][:]
Tokens: ['if', 'n', '==', '0', ':']
----------------------------------------
Input:         return 1
Highlighted Tokens:         [return] [1]
Tokens: ['return', '1']
----------------------------------------
Input:     else:
Highlighted Tokens:     [else][:]
Tokens: ['else', ':']
----------------------------------------
Input:         return n * factorial(n - 1)
Highlighted Tokens:         [retur[[n]]] [[n]] [*] [factor][ial][(][[n]] [-] [1][)]
Tokens: ['ret

In [39]:
with open("tokenized_output.txt", "w") as f:
    for code in code_samples:
        output = bpe_tokenizer.encode(code)
        f.write(f"Input: {code}\n")
        f.write(f"Tokens: {output.tokens}\n")
        f.write(f"Token IDs: {output.ids}\n")
        f.write("-" * 40 + "\n")

In [40]:
import time

def measure_tokenization_speed(tokenizer, file_path):
    with open(file_path, "r") as f:
        code_samples = f.readlines()

    start_time = time.time()
    for code in code_samples:
        tokenizer.encode(code)
    end_time = time.time()

    total_time = end_time - start_time
    print(f"Tokenization Time: {total_time:.2f} seconds")

# Measure speed for BPE tokenizer
measure_tokenization_speed(bpe_tokenizer, "python_code.txt")

Tokenization Time: 25.24 seconds


In [41]:
def calculate_avg_tokens_per_sentence(tokenizer, file_path):
    with open(file_path, "r") as f:
        code_samples = f.readlines()

    total_tokens = 0
    total_samples = len(code_samples)

    for code in code_samples:
        output = tokenizer.encode(code)
        total_tokens += len(output.tokens)

    avg_tokens_per_sentence = total_tokens / total_samples
    print(f"Average Tokens per Sentence: {avg_tokens_per_sentence:.2f}")

# Calculate for BPE tokenizer
calculate_avg_tokens_per_sentence(bpe_tokenizer, "python_code.txt")

Average Tokens per Sentence: 10.97


In [42]:
import tracemalloc

def measure_memory_usage(tokenizer, file_path):
    with open(file_path, "r") as f:
        code_samples = f.readlines()

    tracemalloc.start()
    for code in code_samples:
        tokenizer.encode(code)
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Peak Memory Usage: {peak / 1024:.2f} KB")

# Measure memory usage for BPE tokenizer
measure_memory_usage(bpe_tokenizer, "python_code.txt")

Peak Memory Usage: 524.50 KB


In [43]:
def calculate_oov_rate(tokenizer, file_path):
    with open(file_path, "r") as f:
        held_out_code = f.read()

    output = tokenizer.encode(held_out_code)
    oov_tokens = [token for token in output.tokens if token == "[UNK]"]
    num_oov_tokens = len(oov_tokens)
    total_tokens = len(output.tokens)

    oov_rate = num_oov_tokens / total_tokens
    print(f"OOV Rate: {oov_rate * 100:.2f}%")

# Calculate OOV rate for BPE tokenizer
calculate_oov_rate(bpe_tokenizer, "held_out_python_code.txt")

OOV Rate: 0.00%


In [44]:
def calculate_compression_ratio(tokenizer, file_path):
    with open(file_path, "r") as f:
        code_samples = f.readlines()

    total_chars = 0
    total_tokens = 0

    for code in code_samples:
        output = tokenizer.encode(code)
        total_chars += len(code)
        total_tokens += len(output.tokens)

    compression_ratio = total_chars / total_tokens
    print(f"Compression Ratio: {compression_ratio:.2f}")

# Calculate compression ratio for BPE tokenizer
calculate_compression_ratio(bpe_tokenizer, "python_code.txt")

Compression Ratio: 3.44


In [45]:
def measure_batch_processing_speed(tokenizer, file_path, batch_size=32):
    with open(file_path, "r") as f:
        code_samples = f.readlines()

    start_time = time.time()
    for i in range(0, len(code_samples), batch_size):
        batch = code_samples[i:i + batch_size]
        tokenizer.encode_batch(batch)
    end_time = time.time()

    total_time = end_time - start_time
    print(f"Batch Processing Time: {total_time:.2f} seconds")

# Measure batch processing speed for BPE tokenizer
measure_batch_processing_speed(bpe_tokenizer, "python_code.txt")

Batch Processing Time: 27.60 seconds
